In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
num_col = list(df.describe().columns)
col_categorical = list(set(df.columns).difference(num_col))
remove_list = ['EmployeeCount', 'EmployeeNumber', 'StandardHours']
col_numerical = [e for e in num_col if e not in remove_list]
attrition_to_num = {'Yes': 0,
                    'No': 1}
df['Attrition_num'] = df['Attrition'].map(attrition_to_num)
col_categorical.remove('Attrition')
df_cat = pd.get_dummies(df[col_categorical])
X = pd.concat([df[col_numerical], df_cat], axis=1)
y = df['Attrition_num']

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

C:\Users\asus\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                        lb.transform(clf.predict(X_train)))))

        #cv_res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        #print("Average Accuracy: \t {0:.4f}".format(np.mean(cv_res)))
        #print("Accuracy SD: \t\t {0:.4f}".format(np.std(cv_res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))      
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), lb.transform(res_test))))
        

In [ ]:
# Model 1 ) DT

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [8]:
print_score(tree_clf, X_train, X_test, y_train, y_test, train=True)
print_score(tree_clf, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       179
           1       1.00      1.00      1.00       923

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[179   0]
 [  0 923]]

ROC AUC: 1.0000

Test Result:

accuracy score: 0.7880

Classification Report: 
               precision    recall  f1-score   support

           0       0.35      0.41      0.38        58
           1       0.89      0.86      0.87       310

    accuracy                           0.79       368
   macro avg       0.62      0.64      0.63       368
weighted avg       0.80      0.79      0.79       368


Confusion Matrix: 
 [[ 24  34]
 [ 44 266]]

ROC AUC: 0.6359



In [ ]:
# Model 2 ) RF

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(X_train, y_train.ravel())

RandomForestClassifier()

In [11]:
print_score(rf_clf, X_train, X_test, y_train, y_test, train=True)
print_score(rf_clf, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       179
           1       1.00      1.00      1.00       923

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[179   0]
 [  0 923]]

ROC AUC: 1.0000

Test Result:

accuracy score: 0.8560

Classification Report: 
               precision    recall  f1-score   support

           0       0.67      0.17      0.27        58
           1       0.86      0.98      0.92       310

    accuracy                           0.86       368
   macro avg       0.77      0.58      0.60       368
weighted avg       0.83      0.86      0.82       368


Confusion Matrix: 
 [[ 10  48]
 [  5 305]]

ROC AUC: 0.5781



In [12]:
en_en = pd.DataFrame()

In [13]:
tree_clf.predict_proba(X_train)

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [14]:
en_en['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_train))[1]
en_en['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_train))[1]
col_name = en_en.columns
en_en = pd.concat([en_en, pd.DataFrame(y_train).reset_index(drop=True)], axis=1)

In [15]:
en_en.head()

,tree_clf,rf_clf,Attrition_num
0,1.0,0.99,1
1,1.0,1.00,1
2,1.0,0.94,1
3,1.0,0.96,1
4,1.0,0.89,1


In [16]:
tmp = list(col_name)
tmp.append('ind')
en_en.columns = tmp

In [17]:
en_en.head()

,tree_clf,rf_clf,ind
0,1.0,0.99,1
1,1.0,1.00,1
2,1.0,0.94,1
3,1.0,0.96,1
4,1.0,0.89,1


In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
m_clf = LogisticRegression(fit_intercept=False, solver='lbfgs')

In [20]:
m_clf.fit(en_en[['tree_clf', 'rf_clf']], en_en['ind'])

LogisticRegression(fit_intercept=False)

In [21]:
en_test = pd.DataFrame()

In [22]:
en_test['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_test))[1]
en_test['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_test))[1]
col_name = en_en.columns
en_test['combined'] = m_clf.predict(en_test[['tree_clf', 'rf_clf']])

In [24]:
col_name = en_test.columns
tmp = list(col_name)
tmp.append('ind')

In [25]:
tmp

['tree_clf', 'rf_clf', 'combined', 'ind']

In [26]:
en_test = pd.concat([en_test, pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [27]:
en_test.columns = tmp

In [28]:
print(pd.crosstab(en_test['ind'], en_test['combined']))

combined   0    1
ind              
0         24   34
1         44  266


In [29]:
print(round(accuracy_score(en_test['ind'], en_test['combined']), 4))

0.788


In [30]:
print(classification_report(en_test['ind'], en_test['combined']))

              precision    recall  f1-score   support

           0       0.35      0.41      0.38        58
           1       0.89      0.86      0.87       310

    accuracy                           0.79       368
   macro avg       0.62      0.64      0.63       368
weighted avg       0.80      0.79      0.79       368

